In [ ]:
!pip install earthengine-api

In [ ]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 27.6 MB/s eta 0:00:00


In [ ]:
!pip install mss

In [ ]:
import ee

# Authenticate with Earth Engine (follow the authentication steps)
ee.Authenticate()

# Initialize Earth Engine
ee.Initialize()


In [ ]:
import geemap
from IPython.display import Image


# Create a region of interest (ROI) for India
roi = ee.Geometry.Polygon(
    [[[68.186428335, 6.754758432],
      [97.418835219, 6.754758432],
      [97.418835219, 35.504424046],
      [68.186428335, 35.504424046]]])

# Define a date range
start_date = '2022-01-01'
end_date = '2022-12-31'

# Load a satellite image collection (e.g., Sentinel-2)
collection = (ee.ImageCollection('COPERNICUS/S2')
              .filterBounds(roi)
              .filterDate(ee.Date(start_date), ee.Date(end_date))
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .select(['B4', 'B3', 'B2']))  # Select bands for true color (Red, Green, Blue)

# Get the least cloudy image
image = collection.first()

# Create a visualization parameters dictionary
vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}

# Display the satellite image
Map = geemap.Map()
Map.centerObject(roi, 6)
Map.addLayer(image, vis_params, 'Satellite Image')
Map

# You can also export the image as a PNG or other format if needed
# For example, to export the image as a PNG to your Google Drive:
# export_task = ee.batch.Export.image.toDrive(image=image.clip(roi), description='india_image', folder='earth_engine_exports', scale=10)
# export_task.start()


Map(center=[21.21953118425602, 82.802631777], controls=(WidgetControl(options=['position', 'transparent_bg'], …

In [ ]:
export_task = ee.batch.Export.image.toDrive(image=image.clip(roi),
                                            description='india_image',
                                            folder='earth_engine_exports',
                                            scale=10,
                                            region=roi.getInfo()['coordinates'],
                                            fileFormat='GeoTIFF')
export_task.start()

In [ ]:
import rasterio
import numpy as np

# Load the GeoTIFF image
tif_path = '/content/drive/MyDrive/earth_engine_exports/india_image.tif'
with rasterio.open(tif_path) as src:
    # Read the image data (bands) and metadata
    image_data = src.read()  # This contains multiple bands if available
    image_profile = src.profile  # Metadata, including CRS, geotransform, etc.

# Preprocess the image data (resize, normalize, etc.)
# Example: Normalize pixel values to the range [0, 1]
image_data = (image_data - image_data.min()) / (image_data.max() - image_data.min())

# If you have labels (e.g., for semantic segmentation), load and preprocess them as well

# Now, you can use image_data and labels (if applicable) for your CNN model.


In [ ]:
# Create a region of interest (ROI) for India
roi = ee.Geometry.Polygon(
    [[[68.186428335, 6.754758432],
      [97.418835219, 6.754758432],
      [97.418835219, 35.504424046],
      [68.186428335, 35.504424046]]])
new_delhi_roi = ee.Geometry.Rectangle(77.1, 28.4, 77.4, 28.8)

# Define a date range
start_date = '2022-01-01'
end_date = '2022-12-31'

# Load a satellite image collection (e.g., Sentinel-2)
collection = (ee.ImageCollection('COPERNICUS/S2')
              .filterBounds(new_delhi_roi)
              .filterDate(ee.Date(start_date), ee.Date(end_date))
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .select(['B4', 'B3', 'B2']))  # Select bands for true color (Red, Green, Blue)

# Get the least cloudy image
image = collection.first()

# Create a visualization parameters dictionary
vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}

# Define the number of clusters for segmentation
num_clusters = 5  # Adjust the number of clusters as needed

# Create a smaller subregion within your original ROI
smaller_roi = roi.intersection(ee.Geometry.Rectangle(80, 10, 90, 20))

# Use the smaller_roi when processing the image
feature_collection = image.reduceToVectors(scale=300, geometry=new_delhi_roi, reducer=ee.Reducer.mean())
'''
# Use the "mean" reducer for segmentation
feature_collection = image.reduceToVectors(scale=30, geometry=roi, reducer=ee.Reducer.mean())

# Specify the input properties when training the clusterer
properties = ['0', '1', '2']  # Replace with the actual property names

# Perform clustering-based segmentation
clusterer = ee.Clusterer.wekaKMeans(num_clusters).train(feature_collection, inputProperties=properties)

# Apply the clusterer to the FeatureCollection
segmented_fc = feature_collection.cluster(clusterer)'''

# Specify the input bands for clustering
bands = ['B4', 'B3', 'B2']

# Define a function to compute the mean for each band and set it as a property
def compute_band_means(feature):
    # Reduce the image within each feature to compute the mean for each band
    mean_values = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=300
    )

    # Create a dictionary of properties with band means
    properties = {}
    for band in bands:
        properties[band + '_mean'] = mean_values.get(band)

    return feature.set(properties)

# Map the function over the feature collection
feature_collection_with_means = feature_collection.map(compute_band_means)

# Specify the input properties when training the clusterer
properties = ['B4_mean', 'B3_mean', 'B2_mean']  # Now, you can use the band mean properties

# Perform clustering-based segmentation
clusterer = ee.Clusterer.wekaKMeans(num_clusters).train(feature_collection_with_means, inputProperties=properties)

# Apply the clusterer to the FeatureCollection
segmented_fc = feature_collection_with_means.cluster(clusterer)


# Convert the segmented FeatureCollection back to an image
segmented_image = segmented_fc.randomColumn().reduceToImage(
    properties=['cluster'], reducer=ee.Reducer.first())

# Create a map for visualization
Map = geemap.Map()
Map.centerObject(new_delhi_roi, 6)

num_images = collection.size()
print("Number of images in the collection:", num_images.getInfo())

# Display the satellite image
Map.addLayer(image, vis_params, 'Satellite Image')

# Display the segmented image
Map.addLayer(segmented_image, {'min': 0, 'max': num_clusters, 'palette': ['blue', 'green', 'red', 'yellow', 'purple']}, 'Segmented Image')

# Display the map
Map


Number of images in the collection: 142


Map(center=[28.59995564245501, 77.25000000000023], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
# Convert the map to an Earth Engine image
screenshot = Map.to_image()

# Save the screenshot as a PNG image
screenshot_url = screenshot.getThumbURL({'dimensions': 800, 'format': 'png'})
screenshot_path = 'india_map.png'
geemap.download_ee_video(screenshot_url, screenshot_path)